In [56]:
import json
import pandas as pd
import plotly.express as px

import numpy as np

import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
from flask_babel import gettext
from plotly.subplots import make_subplots
from scipy import stats
import numpy as np
import math
df_prov_tot = pd.read_csv('static/csv/be-covid-provinces-all.csv')
from datetime import datetime, date

#print(df_prov_tot)

# New hospitalizations in each province

In [59]:
def hospi_prov(prov=None):
    df_hospi = df_prov_tot

    if prov is not 'All':
        df_hospi = df_hospi[df_hospi['PROVINCE'] == prov]

    df = df_hospi.groupby(['DATE']).agg({'TOTAL_IN': 'sum', 'NEW_OUT': 'sum', 'NEW_IN': 'sum', 'TOTAL_IN_ICU': 'sum'})

    x2 = df.index
    y2 = df.NEW_IN

    df.index = pd.to_datetime(df.index)
    dw = df.index.dayofweek
    colors_map = {0: '#fccde5', 1: '#8dd3c7', 2: '#b3de69', 3: '#bebada', 4: '#fb8072', 5: '#fdb462', 6: '#80b1d3',
                  7: 'lightgrey'}
    colors = [colors_map[i] for i in dw]

    
    fig = go.Figure(data=[
        go.Bar(x=x2, y=y2, name=gettext('NEW_IN'), legendgroup='group2', showlegend=False, marker_color=colors),
        go.Scatter(x=x2, y=y2.rolling(7,center=True).mean(), showlegend=True, name=gettext('avg past 7 days'),legendgroup='avg 7 days', marker_color="red")
    ])
    start = '2021-01-01' 
    fig.update_layout(yaxis_range=[0,max(y2[x2 >=start])])
    fig.update_layout(xaxis_range=[start, datetime.today().strftime('%Y-%m-%d')])
    fig.update_layout(template="plotly_white", height=300, margin=dict(l=0, r=0, t=30, b=0),title=prov)

    return fig


def hospi_new_in_per_provinces():
    prov = ['All','Liège', 'Brussels','Antwerpen','Namur', 'Luxembourg', 'Hainaut' , 'BrabantWallon', 'VlaamsBrabant', 'OostVlaanderen',
        'WestVlaanderen', 'Limburg',]
    n_cols = 2
    n_rows = math.ceil(1.0 * len(prov)/ n_cols)

    idx = 0
    for prov_name in prov:
            fig = hospi_prov(prov=prov_name)
            
            fig.show()



hospi_new_in_per_provinces()